In [ ]:
import boto3

In [ ]:
ACCESS_KEY = <YOUR_ACCESS_KEY>
# Encode the Secret Key to remove any "/" characters
SECRET_KEY = <YOUR_SECRET_KEY>
AWS_BUCKET_SOURCE = "tweetdump1"
AWS_BUCKET_TARGET = "tableaudump1"
MOUNT_NAME = "/mnt/mount"
DUMP_NAME = "/mnt/dump"

In [ ]:
# try:
#   dbutils.fs.unmount(MOUNT_NAME) # Use this to unmount as needed
# except:
#   print("{} already unmounted".format(MOUNT_NAME))

In [ ]:
try:
  dbutils.fs.mount("s3a://{}:{}@{}".format(ACCESS_KEY, SECRET_KEY, AWS_BUCKET_SOURCE), MOUNT_NAME)
except:
  print("Mount {} already mounted. Run previous cells to unmount first".format(MOUNT_NAME))

In [ ]:
try:
  dbutils.fs.mount("s3a://{}:{}@{}".format(ACCESS_KEY, SECRET_KEY, AWS_BUCKET_TARGET), DUMP_NAME)
except:
  print("Dump {} already mounted. Run previous cells to unmount first".format(DUMP_NAME))

In [ ]:
%fs ls /mnt/mount

In [ ]:
%fs ls /mnt/dump

In [ ]:
%fs head /mnt/mount/tweets.json

In [ ]:
path = '/mnt/mount/tweets.json'
df = (spark.
      read.
      json(path)
)
display(df)

In [ ]:
df.printSchema()

In [ ]:
dfCount = df.count()
dfCount

In [ ]:
from pyspark.sql.types import StructType, StructField, LongType, StringType

tweetSchema = StructType([
  StructField("id", LongType(), True),
  StructField("user", StructType([
    StructField("id", LongType(), True)
  ]), True),  
  StructField("lang", StringType(), True),
  StructField("text", StringType(), True),
  StructField("created_at", StringType(), True)
])

tweetDF = spark.read.schema(tweetSchema).json(path)

display(tweetDF)

In [ ]:
from pyspark.sql.functions import col

tweetFilteredDF = (tweetDF
  .filter(col("id").isNotNull())
)

display(tweetFilteredDF)

In [ ]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.types import TimestampType

timestampFormat = "EEE MMM dd HH:mm:ss ZZZZZ yyyy"
tweetFinalDF = tweetFilteredDF.select(col("id").alias("tweetID"), 
  col("user.id").alias("userID"), 
  col("lang").alias("language"),
  col("text"),
  unix_timestamp("created_at", timestampFormat).cast(TimestampType()).alias("createdAt")
)

display(tweetFinalDF)

In [ ]:
from textblob import TextBlob as tb

In [ ]:
tweetPND = tweetFinalDF.toPandas()
tweetPND["Sentiment"]=tweetPND["text"].apply(lambda x:tb(x).sentiment.polarity)
tweetSentimentDF = spark.createDataFrame(tweetPND)

In [ ]:
tweetSentimentDF.show()

In [ ]:
tweetSentimentDF.write.mode("overwrite").parquet("/mnt/dump/")